In [1]:
import numpy as np
import sys


def abort(var):
    labort = ['exit', 'Exit', 'EXIT', 'abort', 'Abort', 'ABORT', 
             'stop', 'Stop', 'STOP', 'stopp', 'Stopp', 'STOPP']
    if var in labort:
        sys.exit()
    

name = []
hour = []
i = 1

name.append(input('{}{} = '.format(i, ". Name")))
while name[-1] != "0":
    hour.append(float(input('{} = '.format("   Hour"))))
    i += 1
    name.append(input('{}{} = '.format(i, ". Name")))
    
name = name[:-1]
    
tipsum = float(input("total tip = "))

ratio = tipsum / sum(hour)

realtip = np.array([ratio * i for i in hour])
real = np.array([ratio * i for i in hour])

roundtip = np.around(realtip, decimals=1)

if np.around(sum(roundtip), decimals=3) > tipsum:
    print("tip rounded correctly")
    
    check = 5
    
    while np.around(sum(roundtip), decimals=3) > tipsum:
        deci = [int(i * 100) % 10 for i in real]
        hit = [i for i, j in enumerate(deci) if j == check]
        
        for i in range(len(roundtip)):
            if i in hit:
                real[i] = int(real[i] * 10) / 10
                
        roundtip = np.around(real, decimals=1)
        
        check += 1

maxstr = len(max(name, key=len))


print('-' * (maxstr + 28))

for i in range(len(name)):
    print('{num:{width}}'.format(num = name[i], width = maxstr), f' {hour[i]:4.2f}h', f' -> {roundtip[i]:5.1f}€', f' #  {realtip[i]:5.2f}')
    
print('-' * (maxstr + 28))
print(f'total hours = {sum(hour):} h')   
print(f'tip ratio = {ratio:.4} €/h')   
    

KeyboardInterrupt: Interrupted by user

In [27]:
lists=[4.5,4.5,4.5,2.3]
seco = [int(i) for i in lists]
if lists != seco:
    print('hi')

hi


In [34]:
this=float(4)
'{0: <8}|'.format(this)

'4.0     |'

In [1]:

for i in range(len(name)):
    print('{num:{width}}'.format(num = name[i], width = maxstr), f' {hour[i]:4.2f}h', f' -> {roundtip[i]:5.1f}€', f' #  {realtip[i]:5.2f}')
    

NameError: name 'name' is not defined

In [5]:
import sys

def abort(var):
    if var == 'exit':
        print(var)
        
abort('exit')        

exit


In [6]:
import sys

def abort(var):
    if var == 'exit':
        sys.exit()
        
abort('exit') 

SystemExit: 

In [8]:
value = '23.7'
value.replace(',', '.')

'23.7'

In [9]:
3*
4

SyntaxError: invalid syntax (<ipython-input-9-ebc2d024566e>, line 1)

In [9]:
value='3'
float(value)

3.0